### Divide Images + Masks

In [ ]:
import os
from PIL import Image

# Input and Output directories
input_directory = 'ChangeDetectionEurSplit'
output_directory = 'ChangeDetectionEurSplitDivided'

# Function to divide a TIF image into 4 equal quadrants and save them in the output directory
def divide_and_save(image_path, output_dir):
    with Image.open(image_path) as image:
        width, height = image.size

        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Define the coordinates for the 4 quadrants (adjust size if needed)
        quadrants = [
            (0, 0, width // 2, height // 2),       # Top-left
            (width // 2, 0, width, height // 2),  # Top-right
            (0, height // 2, width // 2, height), # Bottom-left
            (width // 2, height // 2, width, height) # Bottom-right
        ]

        # Loop through the quadrants and save them
        for i, coords in enumerate(quadrants, start=1):
            cropped_img = image.crop(coords)

            # Create new file name with suffix _1, _2, etc.
            new_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_{i}.tif"
            new_file_path = os.path.join(output_dir, new_filename)
            cropped_img.save(new_file_path)
            print(f"Saved: {new_file_path}")


# Process the directory structure
for root, dirs, files in os.walk(input_directory):
    for file in files:
        if file.endswith('.tif'):
            # Full path to the input file
            input_file_path = os.path.join(root, file)

            # Calculate the relative path and map it to the output directory
            relative_path = os.path.relpath(root, start=input_directory)
            output_subdir = os.path.join(output_directory, relative_path)

            # Divide and save TIF files
            divide_and_save(input_file_path, output_subdir)

print("Division complete!")


### Divide Image + Masks (if getting `Too moany files open error`)
- Need to run each folder seperately (for train)
- for val and test can run entire folder together

In [ ]:
import os
from PIL import Image

# Input and Output directories
SPLIT = "train"
DIR = "Images/T2019"
# DIR = "Images/T2024"
# DIR = "Masks/T2019"
# DIR = "Masks/T2024"
# DIR = "cd1_Output"
# DIR = "cd2_Output"
# DIR = "cd3_Output"

# SPLIT = "val"
# DIR = ""

# SPLIT = "test"
# DIR = ""

input_directory = f'ChangeDetectionEurSplit/{SPLIT}/{DIR}'
output_directory = f'ChangeDetectionEurSplitDivided/{SPLIT}/{DIR}'

# Function to divide a TIF image into 4 equal quadrants and save them in the output directory
def divide_and_save(image_path, output_dir):
    try:
        image = Image.open(image_path)
        width, height = image.size

        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Define the coordinates for the 4 quadrants
        quadrants = [
            (0, 0, width // 2, height // 2),       # Top-left
            (width // 2, 0, width, height // 2),  # Top-right
            (0, height // 2, width // 2, height), # Bottom-left
            (width // 2, height // 2, width, height) # Bottom-right
        ]

        # Loop through the quadrants and save them
        for i, coords in enumerate(quadrants, start=1):
            cropped_img = image.crop(coords)  # Crop the quadrant
            new_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_{i}.tif"
            new_file_path = os.path.join(output_dir, new_filename)
            cropped_img.save(new_file_path)  # Save the cropped image
            cropped_img.close()  # Close the cropped image
            print(f"Saved: {new_file_path}")

        image.close()  # Close the main image file
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# Process the directory structure
for root, dirs, files in os.walk(input_directory):
    for file in files:
        if file.endswith('.tif'):
            input_file_path = os.path.join(root, file)

            # Calculate the relative path and map it to the output directory
            relative_path = os.path.relpath(root, start=input_directory)
            output_subdir = os.path.join(output_directory, relative_path)

            # Divide and save TIF files
            divide_and_save(input_file_path, output_subdir)

print("Division complete!")

### Verify Divided TIFs 

In [ ]:
import os
import matplotlib.pyplot as plt
import rasterio
import random
import numpy as np

def fetch_and_display_images(real_folder_2019, real_folder_2024,
                             input_folder_2019, input_folder_2024,
                             cd1_output_folder, cd2_output_folder, cd3_output_folder, index):
    """
    Fetch and display images divided into parts: mask_2019, mask_2024, cd1, cd2, cd3, real_2019, real_2024.
    """
    # Find the corresponding real and mask files
    real_2019_file = os.listdir(real_folder_2019)[index]
    # city_state = real_2019_file.replace("_2019.tif", "")
    city_state = real_2019_file[:-11]

    fig, axs = plt.subplots(4, 7, figsize=(25, 15))  # Adjust layout to fit all parts
    fig.suptitle(city_state)
    part_labels = ['1', '2', '3', '4']
    # print(city_state)

    for i, part in enumerate(part_labels):
        # Update file names with parts
        mask_2019_file = f"m_{city_state}_2019_{part}.tif"
        mask_2024_file = f"m_{city_state}_2024_{part}.tif"
        cd1_output = f"cd1_m_{city_state}_{part}.tif"
        cd2_output = f"cd2_m_{city_state}_{part}.tif"
        cd3_output = f"cd3_m_{city_state}_{part}.tif"
        real_2019_file = f"{city_state}_2019_{part}.tif"
        real_2024_file = f"{city_state}_2024_{part}.tif"

        # Paths to files
        mask_2019_path = os.path.join(input_folder_2019, mask_2019_file)
        mask_2024_path = os.path.join(input_folder_2024, mask_2024_file)
        cd1_output_path = os.path.join(cd1_output_folder, cd1_output)
        cd2_output_path = os.path.join(cd2_output_folder, cd2_output)
        cd3_output_path = os.path.join(cd3_output_folder, cd3_output)
        real_2019_path = os.path.join(real_folder_2019, real_2019_file)
        real_2024_path = os.path.join(real_folder_2024, real_2024_file)

        # Ensure files exist
        if not all(os.path.exists(p) for p in [mask_2019_path, mask_2024_path, cd1_output_path, cd2_output_path, cd3_output_path, real_2019_path, real_2024_path]):
            print(f"Some files for part {part} are missing.")
            continue
        # Read and display images for the current part
        with rasterio.open(mask_2019_path) as src:
            mask_2019 = src.read(1)
        with rasterio.open(mask_2024_path) as src:
            mask_2024 = src.read(1)
        with rasterio.open(cd1_output_path) as src:
            cd1_output_mask = src.read(1)
        with rasterio.open(cd2_output_path) as src:
            cd2_output_mask = src.read(1)
        with rasterio.open(cd3_output_path) as src:
            cd3_output_mask = src.read(1)
        with rasterio.open(real_2019_path) as src:
            real_2019 = np.dstack([src.read(band) for band in (1, 2, 3)])
        with rasterio.open(real_2024_path) as src:
            real_2024 = np.dstack([src.read(band) for band in (1, 2, 3)])

        # Display images
        axs[i, 0].imshow(real_2019 / real_2019.max())
        axs[i, 0].set_title(f"Real 2019 {part}")
        axs[i, 0].axis("off")

        axs[i, 1].imshow(real_2024 / real_2024.max())
        axs[i, 1].set_title(f"Real 2024 {part}")
        axs[i, 1].axis("off")

        axs[i, 2].imshow(mask_2019, cmap="viridis")
        axs[i, 2].set_title(f"Mask 2019 {part}")
        axs[i, 2].axis("off")

        axs[i, 3].imshow(mask_2024, cmap="viridis")
        axs[i, 3].set_title(f"Mask 2024 {part}")
        axs[i, 3].axis("off")

        axs[i, 4].imshow(cd1_output_mask, cmap="turbo")
        axs[i, 4].set_title(f"CD1 {part}")
        axs[i, 4].axis("off")

        axs[i, 5].imshow(cd2_output_mask, cmap="turbo")
        axs[i, 5].set_title(f"CD2 {part}")
        axs[i, 5].axis("off")

        axs[i, 6].imshow(cd3_output_mask, cmap="turbo")
        axs[i, 6].set_title(f"CD3 {part}")
        axs[i, 6].axis("off")
    plt.tight_layout()
    plt.show()

# Example usage:
base_folder = "ChangeDetectionEurSplitDivided/train"
real_folder_2019 = f"{base_folder}/Images/T2019"
real_folder_2024 = f"{base_folder}/Images/T2024"
input_folder_2019 = f"{base_folder}/Masks/T2019"
input_folder_2024 = f"{base_folder}/Masks/T2024"
cd1_output_folder = f"{base_folder}/cd1_Output"
cd2_output_folder = f"{base_folder}/cd2_Output"
cd3_output_folder = f"{base_folder}/cd3_Output"

for i in range(10):
    j = random.randint(0, len(os.listdir(input_folder_2019)) - 1)
    fetch_and_display_images(real_folder_2019, real_folder_2024,
                             input_folder_2019, input_folder_2024,
                             cd1_output_folder, cd2_output_folder, cd3_output_folder, j)

## Not used

### Divide CD Outputs

In [ ]:
import os
from PIL import Image

# Base input and output directories
input_directory = './CD'
output_directory = './CDTestDivided'

# Function to divide a TIF image into 4 equal quadrants and save them in the output directory
def divide_and_save(image_path, output_dir):
    image = Image.open(image_path)
    width, height = image.size

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Define the coordinates for the 4 quadrants
    quadrants = [
        (0, 0, width // 2, height // 2),       # Top-left
        (width // 2, 0, width, height // 2),  # Top-right
        (0, height // 2, width // 2, height), # Bottom-left
        (width // 2, height // 2, width, height) # Bottom-right
    ]

    # Loop through the quadrants and save them
    for i, coords in enumerate(quadrants, start=1):
        cropped_img = image.crop(coords)

        # Create new file name with suffix _1, _2, etc.
        new_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_{i}.tif"
        new_file_path = os.path.join(output_dir, new_filename)
        cropped_img.save(new_file_path)
        print(f"Saved: {new_file_path}")


# Iterate over Test, Train, and Val directories
for split in ['Test', 'Train', 'Val']:
    input_split_dir = os.path.join(input_directory, split)
    output_split_dir = os.path.join(output_directory, split)

    # Process cd1_Output, cd2_Output, cd3_Output in each split
    for output_dir in ['cd1_Output', 'cd2_Output', 'cd3_Output']:
        input_subdir = os.path.join(input_split_dir, output_dir)
        output_subdir = os.path.join(output_split_dir, output_dir)

        # Walk through each .tif file in the directory
        for root, dirs, files in os.walk(input_subdir):
            for file in files:
                if file.endswith('.tif'):
                    # Full path to the input file
                    input_file_path = os.path.join(root, file)

                    # Calculate the relative path and map it to the output directory
                    relative_path = os.path.relpath(root, start=input_subdir)
                    target_output_dir = os.path.join(output_subdir, relative_path)

                    # Divide and save the TIF file
                    divide_and_save(input_file_path, target_output_dir)

print("Division of `Test`, `Train`, and `Val` directories complete!")


### View Divided TIFs

In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Path to the original .tif file
# og_file_path = './ChangeDetection/Train/Images/T2019/Alabama_Birmingham_N_2019'
# div_file_path = "./ChangeDetectionDivided/Train/Images/T2019/Alabama_Birmingham_N_2019"
og_file_path = './ChangeDetection/Train/cd1_Output/cd1_m_Alabama_Birmingham_N'
div_file_path = "./ChangeDetectionDivided/Train/cd1_Output/cd1_m_Alabama_Birmingham_N"

# Open the original image
original_image = Image.open(og_file_path + '.tif')
plt.figure(figsize=(15, 5))  # Adjust the figure size as needed

# Plot the original image
plt.subplot(1, 5, 1)  # 1 row, 5 columns, position 1
plt.imshow(np.array(original_image), cmap="viridis", alpha=0.9)
plt.title("Original Image")
plt.axis("off")

# Loop through the 4 quadrants and display each one
for i in range(1, 5):
    quadrant_path = f'{div_file_path}_{i}.tif'
    quadrant_image = Image.open(quadrant_path)
    pixel_values = np.array(quadrant_image)

    # Display the cropped image
    plt.subplot(1, 5, i + 1)
    plt.imshow(pixel_values, cmap="viridis", alpha=0.9)
    plt.title(f"Quadrant {i}")
    plt.axis("off")

plt.tight_layout()
plt.show()
